In [1]:
import time
import sys
sys.path.append('../../dnn/')
import model.ieeg_cnn_rnn
import model.train
import processing.util as util

import numpy as np
# np.random.seed(1234)
from functools import reduce
import math as m
import os

# import DNN frameworks
import tensorflow as tf
import keras
import ntpath

# import high level optimizers, models and layers
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score

# utility for datasets and training
# from keras.utils.training_utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# imports tensorflow
# from keras import backend as K

from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


def poly_decay(epoch, NUM_EPOCHS, INIT_LR):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
    # return the new learning rate
    return alpha

def loadmodel(ieegdnn, **kwargs):
    if model=='cnn':
        # VGG-12 style later
        vggcnn = ieegdnn._build_2dcnn(w_init=w_init, n_layers=n_layers, 
                                      poolsize=poolsize, filter_size=filtersize)
        vggcnn = ieegdnn._build_seq_output(vggcnn, size_fc, DROPOUT)
class Histories(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.losses = []
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        y_pred = self.model.predict(self.model.validation_data[0])
        self.aucs.append(roc_auc_score(self.model.validation_data[1], y_pred))
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return

# 1. Check GPU and System and Load Data

In [3]:
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
# number of gpus
G = 1
print(get_available_gpus())
print(device_lib.list_local_devices())


[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15507112001047907750
]


In [6]:
traindatadir = '/Volumes/ADAM LI/pydata/traindata'
traindatadir = '/home-1/ali39@jhu.edu/scratch/data/dnn/traindata/'
##################### INPUT DATA FOR NN ####################
imagedir = os.path.join(traindatadir, 'image_2d')
# get all the separate files to use for training:
datafiles = []
for root, dirs, files in os.walk(imagedir):
    for file in files:
        datafiles.append(os.path.join(root, file))

sys.stdout.write('\nTraining on ' + str(len(datafiles)) + ' datasets!\n')


Training on 0 datasets!


In [30]:
# train on each data file for some number of epochs
for idx, datafile in enumerate(datafiles):
    filename = path_leaf(datafile)
    metadir = os.path.dirname(datafile)
    if '.npy' in filename:
        filename = filename[:-4]
        print(filename)
    metafile = os.path.join(metadir, 'meta', filename)

    images = np.load(datafile)
    metadata = np.load(metafile)['metadata'].item()

    # load the ylabeled data
    ylabels = metadata['ylabels']
    invert_y = 1 - ylabels
    ylabels = np.concatenate((ylabels, invert_y),axis=1)

#     images = normalizeimages(images) # normalize the images for each frequency band
    
    
    # assert the shape of the images
    assert images.shape[2] == images.shape[3]
    assert images.shape[2] == imsize
    assert images.shape[1] == numfreqs

    print(images.shape)
    print(ylabels.shape)
    
    # format the data correctly 
    # (X_train, y_train), (X_val, y_val), (X_test, y_test) = datahandler.reformatinput(images, labels)
    X_train, X_test, y_train, y_test = train_test_split(images, ylabels, test_size=0.33, random_state=42)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    
    print(images.shape)
    
    break
    # assert ylabels.shape[1]==2
    # assert images.shape

id001_ac_nez1_npz0_0.npz
(479, 4, 32, 32)
(439, 2)


ValueError: Found input variables with inconsistent numbers of samples: [479, 439]

# 2. Load Model We Want

- CNN
- CNN+LSTM
- CNN+LSTM (bidirect)
- 3D CNN
- 3D CNN+LSTM
- 3D CNN+LSTM (bidirect)

In [12]:
imsize=32
numfreqs = 4
numclasses = 2
ieegdnn = model.ieeg_cnn_rnn.IEEGdnn(imsize=imsize, 
                                     n_colors=numfreqs,
                                    num_classes=numclasses)

w_init = None
n_layers = (4,2,1)
poolsize = (2,2)
filtersize = (3,3)

size_fc = 1024
DROPOUT = False #True

# VGG-12 style later
model = ieegdnn._build_2dcnn(w_init=w_init, n_layers=n_layers, 
                              poolsize=poolsize, filter_size=filtersize)
model = ieegdnn._build_seq_output(model, size_fc, DROPOUT)
print(model.summary())

# plot_model(vgg12cnn, to_file='vgg12_plot.png', show_shapes=True, show_layer_names=True)
# SVG(model_to_dot(vgg12cnn).create(prog='dot', format='svg'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 30, 30, 32)        1184      
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 10, 10, 64)        18496     
__________

In [13]:
model = ieegdnn.build_same_cnn_lstm(num_timewins=num_timewins, 
                                  size_mem=size_mem, 
                                  size_fc=size_fc,
                                  BIDIRECT=False,
                                  DROPOUT=DROPOUT)

print(model.summary())
# SVG(model_to_dot(cnn_same_lstm).create(prog='dot', format='svg'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 5, 128)            158208    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              132096    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 2050      
Total params: 423,938
Trainable params: 423,938
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# initialize loss function, SGD optimizer and metrics
loss = 'binary_crossentropy'
optimizer = keras.optimizers.Adam(lr=0.001, 
                                beta_1=0.9, 
                                beta_2=0.999,
                                epsilon=1e-08,
                                decay=0.0)
metrics = ['accuracy']

# compile model
cnn_config = ieegdnn.compile_model(model, 
                                loss=loss, 
                                optimizer=optimizer, 
                                metrics=metrics)

In [15]:
# construct the image generator for data augmentation and construct the set of callbacks
aug = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                            height_shift_range=0.1, 
                                            horizontal_flip=False,
                                            fill_mode="nearest")

# This will do preprocessing and realtime data augmentation:
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    fill_mode='nearest')  # randomly flip images


# 3. Training Procedure

In [20]:
tempdatadir = os.path.join('')
outputdir = os.path.join('')
# checkpoint
filepath=os.path.join(tempdatadir,"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, 
                            monitor='val_acc', 
                            verbose=1, 
                            save_best_only=True, 
                            mode='max')
callbacks = [checkpoint, poly_decay]
INIT_LR = 5e-3
G=1
data_augmentation=False

print(filepath)

weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5


In [ ]:
# augment data, or not and then trian the model!
if not data_augmentation:
    print('Not using data augmentation.')
    HH = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=NUM_EPOCHS,
              validation_data=(X_test, y_test),
              shuffle=False,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    # datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    HH = model.fit_generator(
                datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        shuffle=False,
                        callbacks=callbacks, verbose=2)
print("Finished 1 epoch round")

# 4 Train Each Dataset With Certain Number of Epochs

In [ ]:
modelname = 'cnn'
modeljsonfile = os.path.join(tempdatadir, modelname+"_model.json")
if not os.path.exists(modeljsonfile):
    # serialize model to JSON
    model_json = model.to_json()
    with open(modeljsonfile, "w") as json_file:
        json_file.write(model_json)
    print("Saved model to disk")
        
# train on each data file for some number of epochs
for idx, datafile in enumerate(datafiles):
    filename = path_leaf(datafile)
    metadir = os.path.dirname(datafile)
    if '.npy' in filename:
        filename = filename[:-4]
        print(filename)
        
    data = np.load(datafile)
    image_tensor = data['image_tensor']
    metadata = data['metadata'].item()

    # load the ylabeled data
    ylabels = metadata['ylabels']
    invert_y = 1 - ylabels
    ylabels = np.concatenate((ylabels, invert_y),axis=1)

#     images = normalizeimages(images) # normalize the images for each frequency band
    # assert the shape of the images
    assert images.shape[2] == images.shape[3]
    assert images.shape[2] == imsize
    assert images.shape[1] == numfreqs

    print(images.shape)
    print(ylabels.shape)
    
    # format the data correctly 
    # (X_train, y_train), (X_val, y_val), (X_test, y_test) = datahandler.reformatinput(images, labels)
    X_train, X_test, y_train, y_test = train_test_split(images, ylabels, test_size=0.33, random_state=42)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    
    break
    
    # if tempdatadir follows correct saving procedures
#     weightfiles = natsorted(os.listdir(tempdatadir))
#     recentweightsfile = os.path.join(tempdatadir, weightfiles[-1])
#     model.load_weights(recentweightsfile)
#     print("Loaded model from disk")
    
    # assert ylabels.shape[1]==2
    # assert images.shape
    # augment data, or not and then trian the model!
    if not data_augmentation:
        print('Not using data augmentation.')
        HH = model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=NUM_EPOCHS,
                  validation_data=(X_test, y_test),
                  shuffle=False,
                  callbacks=callbacks)
    else:
        print('Using real-time data augmentation.')
        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        # datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        HH = model.fit_generator(
                    datagen.flow(x_train, y_train, batch_size=batch_size),
                            steps_per_epoch=X_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            shuffle=False,
                            callbacks=callbacks, verbose=2)
    
    # save after each dataset
    model.save(os.path.join(tempdatadir, 
                    'weights-improvement-' + str(idx) + '.h5'))
    print("Saved model weights for ", idx)
        
# save final history object
model.save(os.path.join(outputdatadir, 
                'final_weights' + '.h5'))